In [2]:
import findspark
findspark.init('/home/rajarawal/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lrex_dataset').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True , header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [14]:
output = assembler.transform(data)

In [17]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [19]:
output.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [20]:
final_data = output.select('features','Yearly Amount Spent')

In [21]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

# split dataset into train_data and test_data

In [22]:
train_data , test_data = final_data.randomSplit([0.7 ,0.3])

In [23]:
train_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[30.3931845423455...|  319.9288698031936|
|[30.4925366965402...|  282.4712457199145|
|[30.8364326747734...|  467.5019004269896|
|[30.9716756438877...|  494.6386097568927|
|[31.0613251567161...|  487.5554580579016|
|[31.0662181616375...| 448.93329320767435|
|[31.1239743499119...|  486.9470538397658|
|[31.1695067987115...|  427.3565308022928|
|[31.2834474760581...|  591.7810894256675|
|[31.3091926408918...|  432.7207178399336|
|[31.3123495994443...|  463.5914180279406|
|[31.3584771924370...|  495.1759504494754|
|[31.3895854806643...|  410.0696110599829|
|[31.4252268808548...|  530.7667186547619|
|[31.4459724827577...| 484.87696493512857|
|[31.4474464941278...|   418.602742095224|
|[31.5147378578019...|  489.8124879964614|
|[31.5171218025062...|  275.9184206503857|
|[31.5257524169682...|  443.9656268098819|
|[31.5316044825729...| 436.51560572936256|
+----------

In [24]:
test_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.5743636841713...| 442.06441375806565|
|[30.7377203726281...|  461.7807421962299|
|[30.8162006488763...|   266.086340948469|
|[30.8794843441274...|  490.2065999848547|
|[31.0472221394875...|  392.4973991890214|
|[31.1280900496166...|  557.2526867470547|
|[31.2606468698795...|  421.3266312569514|
|[31.2681042107507...|  423.4705331738239|
|[31.3662121671876...|  430.5888825564849|
|[31.5261978982398...|  409.0945261923378|
|[31.6253601348306...|  376.3369007569242|
|[31.6548096756927...|  475.2634237275485|
|[31.7656188210424...| 496.55408163560713|
|[31.8124825597242...|  392.8103449837972|
|[31.8164283341993...| 501.12249150365636|
|[31.8186165667690...| 446.41867337013565|
|[31.8279790554652...|  440.0027475469415|
|[31.8512531286083...|  472.9922466667984|
|[31.8648325480987...|  439.8912804768137|
+----------

In [25]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                330|
|   mean|  499.7940939149925|
| stddev|  78.57495276081546|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [26]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                170|
|   mean| 498.38216551381055|
| stddev|  80.95762434999007|
|    min|   266.086340948469|
|    max|  744.2218671047146|
+-------+-------------------+



In [27]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [28]:
lr_model = lr.fit(train_data)

In [34]:
print("Coefficients: %s" % str(lr_model.coefficients))
print("Intercept: %s" % str(lr_model.intercept))

Coefficients: [25.407814241699754,38.72102555988245,0.19108556659884,61.55364445409804]
Intercept: -1031.923693790548


In [29]:
test_results = lr_model.evaluate(test_data)

In [30]:
#residuals = difference between predicted label and actual label
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|   9.554304178581333|
|-0.48576836424973635|
|   9.816162512764663|
| -17.676601526608806|
|  -4.419534634505851|
|   4.399149411656595|
|  -7.791096991202039|
| -1.3391811708364685|
|  -4.715916842940487|
|  3.3836364702532364|
|  -9.045954883546244|
|  -4.983386858560891|
|   6.464066108887835|
|  -4.693204001461993|
|  -4.351956915168103|
| -18.088896219142782|
| -2.8059523560911543|
|  -9.519380332542084|
|   7.738906559238387|
| -10.723658960435955|
+--------------------+
only showing top 20 rows



In [31]:
test_results.rootMeanSquaredError

9.866638082204767

In [32]:
test_results.r2

0.9850588129793887

In [33]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



# predictions of unlabled data

In [35]:
unlabled_data = test_data.select('features')

In [36]:
unlabled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.5743636841713...|
|[30.7377203726281...|
|[30.8162006488763...|
|[30.8794843441274...|
|[31.0472221394875...|
|[31.1280900496166...|
|[31.2606468698795...|
|[31.2681042107507...|
|[31.3662121671876...|
|[31.5261978982398...|
|[31.6253601348306...|
|[31.6548096756927...|
|[31.7656188210424...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8186165667690...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.8648325480987...|
+--------------------+
only showing top 20 rows



In [37]:
predictions = lr_model.transform(unlabled_data)

In [38]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|399.08604689404615|
|[30.5743636841713...| 442.5501821223154|
|[30.7377203726281...| 451.9645796834652|
|[30.8162006488763...| 283.7629424750778|
|[30.8794843441274...| 494.6261346193605|
|[31.0472221394875...| 388.0982497773648|
|[31.1280900496166...| 565.0437837382567|
|[31.2606468698795...|422.66581242778784|
|[31.2681042107507...| 428.1864500167644|
|[31.3662121671876...| 427.2052460862317|
|[31.5261978982398...|418.14048107588405|
|[31.6253601348306...| 381.3202876154851|
|[31.6548096756927...| 468.7993576186607|
|[31.7656188210424...| 501.2472856370691|
|[31.8124825597242...| 397.1623018989653|
|[31.8164283341993...| 519.2113877227991|
|[31.8186165667690...| 449.2246257262268|
|[31.8279790554652...| 449.5221278794836|
|[31.8512531286083...|   465.25334010756|
|[31.8648325480987...|450.61493943724963|
+--------------------+------------